In [ ]:
### code to segregate photos without text into a folder so that photos only with faces are in a separate folder, 
###fully automated solution to create and move folders and files along with respective profiles of depressed and happy users. 

# using the tesseract library to detect text and move it into a separate folder. 
import cv2
import pytesseract
import shutil, os
import cv2
import glob
import ntpath
from pathlib import Path
import numpy as np
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
for filepath in glob.iglob(r'C:\Users\joshm\instaprofs\**\*.jpg',recursive=True):
    yo = os.path.basename(os.path.dirname(filepath))
    yo = yo + "_notext\\"
    n = r'C:\Users\joshm' + "\\"
    path = os.path.join(n, yo)
    img = cv2.imread(filepath)
    text = pytesseract.image_to_string(img)

    # if there is no text, a folder with the profile name is created and corresponding files are moved into that folder. 
    if len(text) == 0:
        text_count = 0
        if not os.path.isdir(os.path.basename(os.path.dirname(path))):
            os.mkdir(path)
            shutil.move(filepath, path)
        else:
            shutil.move(filepath, path)
    else:
        text_count = 1


In [ ]:
### code to read files from folders and subfolders and find out hsv and facecount values. 
### this is run only on folders with images containing faces after the above code segregates photos with faces and no faces. 

# import required libraries and packages. 
import glob
import ntpath
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os

# set counter to 0 to count total files to loop through and start loop
count = 0

# loop to find facecount, hue , saturation and brighness value all in one go for directories and subdirectories 
# and add the filename, hsv values and face count values to a dataframe for dataset processing. 

for filepath in glob.iglob(r'C:\Users\joshm\Downloads\clean-20200406T012428Z-001\clean\*.jpg'):
    count +=1
cols = ['Filename', 'Hue', 'Saturation', 'Brightness', 'FaceCount']
df2 = pd.DataFrame(columns=cols, index=range(count))
count = 0
for filepath in glob.iglob(r'C:\Users\joshm\Downloads\clean-20200406T012428Z-001\clean\*.jpg'):
    filename = ntpath.basename(filepath)
    yo = os.path.basename(os.path.dirname(filepath))
    yo = yo + "# "
    filename = yo + filename
    face_cascade = cv2.CascadeClassifier(r'C:\Users\joshm\haarcascade_frontalface_default.xml')
    # opencv reads an image in BGR scale, converting it to grayscale for hsv processing. 
    img = cv2.imread(filepath)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.05, 5)
    # Detect smile
    smiles = smile_cascade.detectMultiScale(gray, 1.8, 20)
    smiles = np.asarray(smiles)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # finding the mean of all pixels in the image and extracting the mean of hsv from the image. 
    hue = hsv[...,0].mean()
    sat = hsv[...,1].mean()
    bright = hsv[...,2].mean()
    # if there is no face, it means it is a shot of nature/buildings , so assigning facecount to zero. 
    if len(faces) == 0:
        face_count = 0
    else:
        face_count = faces.shape[0]
    if smiles1.size == 0:
        smile = 0
    else:
        smile = 1
    df2.loc[count].Filename = filename
    df2.loc[count].Hue = hue
    df2.loc[count].Saturation = sat
    df2.loc[count].Brightness = bright
    df2.loc[count].FaceCount = face_count
    count +=1

In [ ]:
### code to group records by profile and find the mean, max and min for each of hue, saturation and brightness value 
### of all images so that there is only one record corresponding to one user combining all images of that particular user. 


# using # as the delimiter to split the user names and their respective file names. 
grouped = df2.groupby(df2['Filename'].str.split('#').str[0])
grouped.groups
totalphotos = grouped.size()
totalphotos = totalphotos.to_frame()
df2 = df2.join(totalphotos)
df2[['Hue', 'Saturation', 'Brightness','FaceCount','SmileYes']] = df2[['Hue', 'Saturation', 'Brightness', 'FaceCount','SmileYes' ]].apply(pd.to_numeric)
# finding mean max and min for each profile to render one record per user in happy and depressed category.
face = df2.groupby(df2['Filename'].str.split('#').str[0]).agg({ 
    'Hue':['mean', 'max', 'min'],
    'Saturation':['mean','max', 'min'],
    'Brightness':['mean','max', 'min'],
    'FaceCount' :['mean','max', 'min'],
    'SmileYes' :['mean']
})
face.columns = ['HueMean', 'HueMax', 'HueMin','SaturationMean', 'SaturationMax', 
                   'SaturationMin','BrightnessMean', 'BrightnessMax', 'BrightnessMin', 'FaceCountMean', 'FaceCountMax', 
                     'FaceCountMin', 'SmileYes', 'TotalPhotos']
face.to_csv(r'C:\Users\joshm\imagedata2.1.csv')